## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Teddy-note-tutorial")

LangSmith 추적을 시작합니다.
[프로젝트명]
Teddy-note-tutorial


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [4]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [5]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [6]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [7]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [9]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [10]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [11]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 데이터를 입력으로 받아 이를 분석하고 패턴을 학습하는 과정입니다. 모델은 입력된 데이터를 특정한 방식으로 처리하여 원하는 결과를 예측하거나 분류하는데 사용됩니다.\n\n일반적으로, 인공지능 모델은 입력 데이터를 받아 가중치와 편향을 조절하면서 학습을 진행합니다. 모델은 입력 데이터와 정답(레이블) 사이의 차이를 최소화하는 방향으로 가중치를 조정하며, 이를 위해 손실 함수와 옵티마이저 등의 기법을 사용합니다.\n\n학습 과정은 여러 번의 반복을 통해 이루어지며, 모델이 입력 데이터를 보다 정확하게 예측하도록 가중치를 조정하면서 성능을 향상시킵니다. 이러한 과정을 통해 모델은 학습 데이터에서 학습한 패턴을 새로운 데이터에 적용하여 원하는 결과를 예측하거나 분류할 수 있습니다.', response_metadata={'token_usage': {'completion_tokens': 330, 'prompt_tokens': 33, 'total_tokens': 363}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9081a44d-8762-4481-84f4-39469d331409-0', usage_metadata={'input_tokens': 33, 'output_tokens': 330, 'total_tokens': 363})

아래는 스트리밍을 출력하는 예시 입니다.

In [12]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력받아 패턴을 학습하고 이를 기반으로 예측이나 결정을 내리는 과정입니다. 

먼저, 모델은 입력된 데이터를 기반으로 가중치를 조절하고 편향을 추가하여 학습을 진행합니다. 이를 통해 모델은 데이터의 패턴을 파악하고 예측을 수행할 수 있게 됩니다. 

학습 과정에서 모델은 손실 함수를 이용하여 예측값과 실제값의 차이를 계산하고, 이를 최소화하기 위해 가중치를 업데이트합니다. 이를 반복하여 모델이 최적의 가중치와 편향을 찾아내게 됩니다. 

이렇게 학습된 모델은 새로운 데이터를 입력받아 패턴을 파악하고 예측을 수행할 수 있습니다.

### 출력파서(Output Parser)


In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [14]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [15]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 데이터를 사용하여 모델이 패턴을 학습하는 과정입니다. 모델은 입력 데이터를 받아들이고, 내부의 가중치와 편향을 조정하여 원하는 출력을 만들어 내도록 학습됩니다. \n\n이때, 모델은 입력 데이터와 정답 데이터를 사용하여 손실 함수를 계산하고, 이 손실 함수를 최소화하는 방향으로 학습을 진행합니다. 이를 위해 경사 하강법과 같은 최적화 알고리즘이 사용되어 모델의 가중치와 편향을 업데이트하게 됩니다. \n\n이런 과정을 반복하여 모델이 입력 데이터를 올바르게 예측할 수 있도록 학습시키는 것이 인공지능 모델의 핵심 원리입니다.'

In [16]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 활용하여 모델이 패턴이나 규칙을 학습하도록 하는 것입니다. 모델은 입력 데이터를 받아들이고, 이를 처리하여 원하는 결과를 출력합니다. 

모델은 초기에는 무작위로 설정된 가중치와 편향을 가지고 있어서 정확한 결과를 출력하지 못합니다. 이때, 학습 데이터를 이용하여 모델의 가중치와 편향을 조정하면서, 입력 데이터와 출력 결과 사이의 관계를 학습하게 됩니다. 

이 과정에서 모델은 손실 함수를 통해 예측 결과와 실제 결과의 차이를 계산하고, 이 손실을 최소화하는 방향으로 가중치와 편향을 조정하게 됩니다. 이러한 과정을 반복하면서 모델은 학습 데이터에 대해 높은 정확도를 가지는 패턴이나 규칙을 학습하게 되는 것이죠.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [17]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4-turbo")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [18]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [19]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  Customer: "Hi, I'd like to have a table for two, please."
  Waiter: "Certainly! Right this way. Here's your menu."
  Customer: "Thank you. We'll need a couple of minutes to decide."
  Waiter: "Take your time. Can I start you off with some drinks?"
  Customer: "Yes, could we have two glasses of water and one orange juice, please?"
  Waiter: "Sure thing. I'll bring those right over. Let me know when you're ready to order."

- 한글 해석:
  손님: "안녕하세요, 두 명이서 식사할 수 있는 테이블을 부탁해요."
  웨이터: "물론입니다! 이쪽으로 오세요. 여기 메뉴판입니다."
  손님: "감사합니다. 주문할 메뉴를 결정하는 데 조금 시간이 필요할 것 같아요."
  웨이터: "천천히 결정하세요. 음료로 먼저 시작하시겠어요?"
  손님: "네, 물 두 잔과 오렌지 주스 한 잔 주세요."
  웨이터: "알겠습니다. 곧 가져다 드릴게요. 주문할 준비가 되면 알려주세요."


In [20]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  A: Hi, could I have a table for two, please?
  B: Of course! Right this way, please.
  A: Thank you. We'd like to order now. Could I get the grilled salmon, and could she have the beef stew?
  B: Absolutely, would you like any drinks or starters?
  A: Yes, we'll have two glasses of white wine and a Caesar salad to share.
  B: Great choices! I'll get those orders in for you.

- 한글 해석:
  A: 안녕하세요, 두 명이서 식사할 수 있는 자리 있나요?
  B: 물론입니다! 이쪽으로 오세요.
  A: 감사합니다. 지금 주문하고 싶어요. 저는 그릴 연어를 주문할게요, 그리고 그녀는 소고기 스튜를 주문할래요.
  B: 좋습니다, 음료나 전채 요리도 주문하시겠어요?
  A: 네, 화이트 와인 두 잔과 시저 샐러드 하나를 공유할게요.
  B: 좋은 선택이세요! 주문을 넣어 드리겠습니다.

In [21]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  Person: Hi, I’d like to order a pizza for delivery, please.
  Staff: Sure, what would you like?
  Person: I’d like a large pepperoni pizza with extra cheese.
  Staff: Anything else?
  Person: Yes, can I also have a side of garlic bread and a 2-liter bottle of Coke?
  Staff: Absolutely. Can I have your address, please?
  Person: It’s 452 Pine Street, Apartment 2B.
  Staff: Great, it'll be there in about 30-45 minutes. Your total comes to $29.50.
  Person: Perfect, thank you!

- 한글 해석:
  고객: 안녕하세요, 배달로 피자 하나 주문하고 싶어요.
  직원: 네, 무엇을 드릴까요?
  고객: 큰 사이즈 페퍼로니 피자에 치즈를 추가해 주세요.
  직원: 추가로 더 필요하신 것이 있나요?
  고객: 네, 마늘빵 하나와 콜라 2리터도 주세요.
  직원: 알겘습니다. 주소를 알려주시겠어요?
  고객: 452 Pine Street, Apartment 2B입니다.
  직원: 좋습니다. 약 30-45분 내에 배달될 거에요. 총 금액은 $29.50입니다.
  고객: 좋습니다, 감사합니다!